In [75]:
#подгружаю пакетики с запасом
import numpy as np
import math
import sympy as sy
from scipy.misc import derivative
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from scipy.stats import chi2_contingency

In [65]:
# Загружаю данные, называю их 'mydata'. Обращаю внимание на параметр функции read_csv (из пакета pandas): decimal=",". 
# В исходном файле  десятичный разделитель - это запятая. Питон привык думать, что запятая должна разделять что угодно, но не целую часть
# числа от дробной. Этот параметр изменяет запятые в цифрах на точки. Теперь питон поймет, что перед ним число, а не какой-то объект.

mydata = pd.read_csv('C:\\Users\\alina\\Downloads\\pca2.csv',sep=';', engine='python', header=None, na_values='?',decimal=",",delimiter=";")

In [66]:
# Посмотрим на наш датасет. Жалко, что без имен ):
mydata

,0,1,2,3,4
0,68.9,1060,7.8,5.5,25.3
1,68.1,1101,9.5,15.3,28.0
2,67.6,1147,10.1,30.2,30.0
3,69.2,1204,10.0,44.5,23.5
4,69.2,1602,9.8,58.6,18.0
5,64.6,1893,5.5,93.3,38.4
6,67.0,2777,6.2,122.0,29.6


In [88]:
# Перед тем, как создавать функцию, оценим наши шансы на применимость PCA. 

#Строим корреляционную матрицу 
corr_matrix = mydata.corr()
print(corr_matrix)

print('Определитель корреляционной матрицы равен:')
print(np.linalg.det(corr_matrix))

          0         1         2         3         4
0  1.000000 -0.500840  0.769223 -0.595292 -0.934048
1 -0.500840  1.000000 -0.700093  0.953664  0.296602
2  0.769223 -0.700093  1.000000 -0.669180 -0.680883
3 -0.595292  0.953664 -0.669180  1.000000  0.370110
4 -0.934048  0.296602 -0.680883  0.370110  1.000000
Определитель корреляционной матрицы равен:
0.0008300750172535987


In [115]:
#По матрице, в целом, видно, что многие переменные зависимы. Но код должен быть универсальным.
# Запустим цикл: пусть программа сама ищет в корреляционной матрице значения, по модулю большие, чем хотя бы 0.8
# Отметим, что статистики принимают гипотезу о зависимости 2х факторов, если r>0.3, но мы осуществим более строгий отбор.
#Таким образом, программа выдаст нам свои предположения о возможности применения PCA. Но это еще не конец.

for i in range(len(corr_matrix)):
    for j in range(len(corr_matrix)):
        if abs(corr_matrix[i][j]) > 0.8:
            print('Некоторые переменные взаимосвязаны. Возможно, стоит применить РСА.')
            break
        else: 
            print('Нет взаимосвязанных переменных. Возможно, не стоит применить РСА.')
    break       

Некоторые переменные взаимосвязаны. Возможно, стоит применить РСА.


In [208]:
#ТЕСТ СФЕРИЧНОСТИ БАРЛЕТТА

#Создадим функцию от переменной - наш датасет (любой). Эта фунция вычисляет критерий Хи-квадрат для нашего датасета и сравнивает его 
# с критическим значением. Итог: программа скажет, можно ли применять PCA и почему. Также я оставила вывод расчетов критерия:
#chi2 (test Statistics) - это значение самого критерия. df(degrees of freedom) - это число степеней сводобы. p_value - значимость нашего критерия.

def Barlet_krit(data):
    data = mydata
    chi2, p_value, df, expected = chi2_contingency(data)
    output = "test Statistics: {}\ndegrees of freedom: {}\np-value: {}\n"
    if prob<0.05:
        result = print(output.format( chi2, df, p_value),'Нулевая гипотеза отклоняется на уровне 5%. Можно эффективно применить PCA для исследуемого набора данных. p_value=' , p_value, '<0.05')
    else:
        result =print(output.format( chi2, df, p_value),'Нулевая гипотеза отвергается на уровне 5%. Не рекомендуется применение PCA для исследуемого набора данных. p_value=', p_value, '>0.05')
    return result

# Проверим, как работает наша функция (как надо, убедились)
Barlet_krit(mydata)

test Statistics: 156.28817741484394
degrees of freedom: 24
p-value: 2.2285372917153736e-21
 Нулевая гипотеза отклоняется на уровне 5%. Можно эффективно применить PCA для исследуемого набора данных. p_value= 2.2285372917153736e-21 <0.05


In [170]:
# КРИТЕРИЙ КМО
#Если нужна функция - см. конец. Здесь я буду объяснять куски своего кода, чтобы функция потом по ночам не снилась.

#Подготовим данные для нашей функции. Для начала, с помощью метода inv из пакета numpy вычислим обратную матрицу корреляций.
corr_matrix_inv=np.linalg.inv(corr_matrix)

In [177]:
#Создаем пустой лист, куда на каждой итерации будет добавляться новое значение.
vector_chastn_corr=[]

#А вот и цикл. Здесь мы по определению счиатем матрицу частных корреляций (тк в питоне нет встроенной функции для ее расчета).
# Теперь наша переменная, которую мы создали ранее - vector_chastn_corr - наполнилась значениями матрицы частных корреляций. Но есть проблема.
for k in range(len(corr_matrix_inv)):
    for l in range(len(corr_matrix_inv)):
        vector_chastn_corr.append(-(corr_matrix_inv[k][l]/np.sqrt(corr_matrix_inv[k][k]*corr_matrix_inv[l][l])))

In [178]:
# Здесь решим эту проблему. Наша матрица пока еще не матрица, а список. Сначала сделаем ее вектором:
vector_chastn_corr=np.array(vector_chastn_corr)

# А теперь из вектора соберем матрицу нужного размера (такого же, как и обратная матрица)
martix_chastn_corr=vector_chastn_corr.reshape(corr_matrix_inv.shape)

In [179]:
# Посмотрим на нашу матрицу частных корреляций. Аномалий нет, можно работать дальше.
martix_chastn_corr

array([[-1.        ,  0.40370415,  0.30446504, -0.60281488, -0.90574968],
       [ 0.40370415, -1.        , -0.58399811,  0.92064463,  0.17107425],
       [ 0.30446504, -0.58399811, -1.        ,  0.40912531, -0.01735923],
       [-0.60281488,  0.92064463,  0.40912531, -1.        , -0.40569733],
       [-0.90574968,  0.17107425, -0.01735923, -0.40569733, -1.        ]])

In [219]:
#Создаем снова списки-накопители результатов каждой итерации - для элементов корреляционной матрицы
# И для элементов матрицы частных корреляций. Потом найдем их сумму.
sum_corr=[]
sum_chastn =[]

#Добавляем в списки квадрат каждого элемента каждой матрицы (кроме тех, где i=j) в созданные выше списки
# Суммируем элементы каждого списка
for i in range(len(corr_matrix_inv)):
    for j in range(len(martix_chastn_corr)):
        if i != j:
            sum_corr.append (corr_matrix[i][j]**2)
            sum_chastn.append(martix_chastn_corr[i][j]**2)
#считаем критерий КМО. Как видим, он хорош, значит функция должна разрешить нам применить PCA. Посмотрим(:         
sum(sum_corr)/(sum(sum_corr)+sum(sum_chastn))

0.6063666429300164

In [220]:
# Это функция. Все ее элементы кода расписаны выше. Я просто здесь собрала пазл из кусочков моего кода. 

def krit_KMO(mydata):
    corr_matrix = mydata.corr()
    corr_matrix_inv=np.linalg.inv(corr_matrix)
    vector_chastn_corr=[]
    for k in range(len(corr_matrix_inv)):
        for l in range(len(corr_matrix_inv)):
            vector_chastn_corr.append(-(corr_matrix_inv[k][l]/np.sqrt(corr_matrix_inv[k][k]*corr_matrix_inv[l][l])))
    vector_chastn_corr=np.array(vector_chastn_corr)
    martix_chastn_corr=vector_chastn_corr.reshape(corr_matrix_inv.shape)
    sum_corr=[]
    sum_chastn =[]
    for i in range(len(corr_matrix_inv)):
        for j in range(len(martix_chastn_corr)):
            if i != j:
                sum_corr.append (corr_matrix[i][j]**2)
                sum_chastn.append(martix_chastn_corr[i][j]**2)
    result =  sum(sum_corr)/(sum(sum_corr)+sum(sum_chastn))
    if result >0.6:
        output = print('Метод PCA применим, так как индекс КМО =', result)
    else:
        output = print('Метод PCA не применим, так как индекс КМО =', result)
    return output

In [221]:
# Проверим, как работает:
krit_KMO(mydata)

Метод PCA применим, так как индекс КМО = 0.6063666429300164
